In [ ]:
!pip install nibabel nilearn matplotlib nilearn[plotting] --quiet
!pip install bids-validator --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting, image, masking
from nilearn.glm.first_level import FirstLevelModel
from bids_validator import BIDSValidator
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nilearn.glm.first_level import FirstLevelModel
from nilearn import image, masking, plotting


In [ ]:
def validate_bids_structure(bids_path):
    validator = BIDSValidator()
    is_valid = validator.is_bids(bids_path)
    print(f"BIDS validation for {bids_path}: {'Valid' if is_valid else 'Invalid'}")
    return is_valid

In [ ]:
def load_nifti(file_path):
    print(f"Loading NIfTI file: {file_path}")
    return nib.load(file_path)

In [ ]:
def smooth_img(func_img, fwhm=6):
    print("Applying smoothing...")
    return image.smooth_img(func_img, fwhm)

def standardize_img(func_img):
    print("Standardizing functional image (z-scoring)...")
    data = func_img.get_fdata()
    mean = np.mean(data, axis=-1, keepdims=True)
    std = np.std(data, axis=-1, keepdims=True)
    standardized_data = (data - mean) / std
    return nib.Nifti1Image(standardized_data, affine=func_img.affine)

def apply_mask(func_img, mask_img=None):
    print("Applying brain mask...")
    return masking.apply_mask(func_img, mask_img)

In [ ]:
# def plot_2d_slices(img, title="Scan Slices"):
#     print("Displaying 2D slices...")
#     plotting.plot_anat(img, title=title, display_mode='ortho', draw_cross=True)

def plot_3d_glassbrain(img, title="3D Glass Brain"):
    print("Displaying 3D glass brain...")
    plotting.plot_glass_brain(img, title=title)

# def plot_overlay(anat_img, func_img, title="Functional Overlay"):
#     print("Overlaying functional on anatomical...")
#     plotting.plot_anat(anat_img, title=title, display_mode='ortho')
#     plotting.plot_stat_map(func_img, bg_img=anat_img, threshold=1, title=title)

In [ ]:
def plot_2d(img, title=""):
    """
    Plots a 3D anatomical or functional brain image.
    If a 4D image is passed (e.g., fMRI), it will use the first timepoint.
    """
    if img.ndim == 4:
        img = image.index_img(img, 0)
    plotting.plot_anat(img, title=title, display_mode='ortho', draw_cross=True)

def plot_overlay(anat_img, func_img, title="Overlay"):
    """
    Plots a functional image over an anatomical background.
    Accepts 3D or 4D functional images. Defaults to first timepoint for 4D.
    """
    if func_img.ndim == 4:
        func_img = image.index_img(func_img, 0)
    plotting.plot_stat_map(func_img, bg_img=anat_img, title=title, threshold=1.0)


In [ ]:
def run_glm(func_img, events_path, t_r=2.0):
    print("Running First-Level GLM...")
    fmri_glm = FirstLevelModel(t_r=t_r, noise_model='ar1', standardize=True, hrf_model='spm')
    fmri_glm = fmri_glm.fit(func_img, events_file=events_path)
    z_map = fmri_glm.compute_contrast('task', output_type='z_score')
    return z_map

In [ ]:
from nilearn.masking import compute_epi_mask
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting

In [ ]:
mask_img = compute_epi_mask(func_img_smooth)

In [ ]:
def run_glm_and_plot(func_img, anat_img, events_file, t_r=2.0, title="GLM Activation (Z-map)"):
    """
    Run a GLM on fMRI data using the given events file (tsv) and visualize the resulting z-map.
    """

    events_df = pd.read_csv(events_file, sep='\t')
    print("Loaded events file:")
    print(events_df.head())

    print("Fitting first-level GLM...")
    model = FirstLevelModel(
    t_r=t_r,
    noise_model='ar1',
    standardize=True,
    hrf_model='spm',
    smoothing_fwhm=None,
    mask_img=mask_img
)
    model = model.fit(func_img, events=events_df)

    design = model.design_matrices_[0]
    print(f"Available contrasts: {design.columns.tolist()}")


    contrast = design.columns[1] if len(design.columns) > 1 else design.columns[0]
    print(f"Computing contrast: {contrast}")


    z_map = model.compute_contrast(contrast, output_type='z_score')


    print("Displaying Z-map...")
    plotting.plot_stat_map(z_map, bg_img=anat_img, title=title, threshold=3.0)
    return z_map


In [ ]:
base_path = "/content/drive/My Drive/fMRI Data/fMRI/sub-216"

anat_path = os.path.join(base_path, "anat", "sub-216_T1w.nii.gz")
func_path = os.path.join(base_path, "func", "sub-216_task-shapessocial_bold.nii.gz")
event_path = os.path.join(base_path, "func", "sub-216_task-shapessocial_events.tsv")

anat_img = load_nifti(anat_path)
func_img = load_nifti(func_path)

func_img_smooth = smooth_img(func_img)
func_img_z = standardize_img(func_img_smooth)

    # Visualize
# plot_2d_slices(anat_img, "Anatomical Scan")
# plot_2d_slices(func_img_z, "Functional (z-scored)")
# plot_overlay(anat_img, func_img_z, "Overlay")
plot_2d(anat_img, "Anatomical Scan")

In [ ]:
plot_2d(func_img_z, "Functional (z-scored, t=0)")

In [ ]:
plot_overlay(anat_img, func_img_z, "Functional Overlay")

In [ ]:
z_map = run_glm_and_plot(func_img_z, anat_img, event_path)
z_map

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from nilearn import plotting, image
import ipywidgets as widgets
from IPython.display import display

def show_timepoint_slider(img_4d, bg_img=None):
    """
    Creates an interactive slider to scroll through timepoints in a 4D image.
    """
    n_timepoints = img_4d.shape[-1]

    def plot_timepoint(t):
        display(image.index_img(img_4d, t))
        plotting.plot_stat_map(
            image.index_img(img_4d, t),
            bg_img=bg_img,
            threshold=1.0,
            title=f"Timepoint {t}",
            display_mode='ortho',
            cut_coords=(0, 0, 0)
        )

    slider = widgets.IntSlider(min=0, max=n_timepoints-1, step=1, description='Time:')
    widgets.interact(plot_timepoint, t=slider)


In [ ]:
def show_mean_fmri(func_img_4d, anat_img=None, title="Mean Functional Image"):
    """
    Calculates and plots the mean image across all timepoints in a 4D functional scan.
    """
    print("Computing mean image across time...")
    mean_img = image.mean_img(func_img_4d)
    plotting.plot_stat_map(mean_img, bg_img=anat_img, title=title, threshold=1.0)
    return mean_img

In [ ]:
show_timepoint_slider(func_img_z, bg_img=anat_img)

In [ ]:
from nilearn.masking import compute_epi_mask

def interactive_glm_contrast_viewer(func_img_glm, func_img_for_mask, anat_img, events_path, t_r=2.0):
    """
    Fits a GLM using an explicit mask and allows interactive visualization of contrasts.
    func_img_glm: preprocessed fMRI for GLM (e.g., z-scored or smoothed)
    func_img_for_mask: original or smoothed 4D fMRI (not z-scored) for computing brain mask
    """

    events_df = pd.read_csv(events_path, sep='\t')

    print("Computing brain mask from non-zscored image...")
    mask_img = compute_epi_mask(func_img_for_mask)

    print("Fitting GLM with explicit mask...")
    model = FirstLevelModel(
        t_r=t_r,
        standardize=True,
        hrf_model='spm',
        mask_img=mask_img
    )
    model = model.fit(func_img_glm, events=events_df)

    design = model.design_matrices_[0]
    contrast_names = design.columns.tolist()

    def plot_contrast(contrast_name):
        z_map = model.compute_contrast(contrast_name, output_type='z_score')
        plotting.plot_stat_map(
            z_map,
            bg_img=anat_img,
            title=f"Z-map for Contrast: {contrast_name}",
            threshold=3.0,
            display_mode='ortho',
            cut_coords=(0, 0, 0)
        )

    widgets.interact(plot_contrast, contrast_name=widgets.Dropdown(options=contrast_names))


In [ ]:
interactive_glm_contrast_viewer(
    func_img_glm=func_img_z,
    func_img_for_mask=func_img_smooth,
    anat_img=anat_img,
    events_path=event_path
)


In [ ]:
mean_func = show_mean_fmri(func_img_z, anat_img=anat_img)

In [ ]:
def top_voxel_fits(model, func_img, top_n=5):
    """
    Plot GLM fits for top N voxels with highest model R².
    """
    mask_img = model.mask_img_
    masked_data = masking.apply_mask(func_img, mask_img)
    predicted = model.predicted[0]

    residuals = masked_data - predicted
    rss = np.sum(residuals ** 2, axis=0)
    tss = np.sum((masked_data - masked_data.mean(axis=0)) ** 2, axis=0)
    r_squared = 1 - (rss / tss)

    best_voxels = np.argsort(-r_squared)[:top_n]
    print(f"Top {top_n} voxel indices (in mask space):", best_voxels)

    for i, idx in enumerate(best_voxels):
        plt.figure(figsize=(10, 3))
        plt.plot(model.frame_times, masked_data[:, idx], label='Observed', marker='o')
        plt.plot(model.frame_times, predicted[:, idx], label='Predicted', linestyle='--')
        plt.title(f"Top-{i+1} Fit (Voxel Index in Mask: {idx}) - R²: {r_squared[idx]:.3f}")
        plt.xlabel("Time (s)")
        plt.ylabel("BOLD Signal")
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display

def interactive_voxel_fit(model, func_img):
    x_widget = widgets.IntSlider(min=0, max=func_img.shape[0]-1, description='x')
    y_widget = widgets.IntSlider(min=0, max=func_img.shape[1]-1, description='y')
    z_widget = widgets.IntSlider(min=0, max=func_img.shape[2]-1, description='z')

    def update(x, y, z):
        plot_voxel_fit(model, (x, y, z), func_img)

    display(widgets.interactive(update, x=x_widget, y=y_widget, z=z_widget))


In [ ]:
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from nilearn.masking import compute_epi_mask


events_df = pd.read_csv(event_path, sep='\t')


mask_img = compute_epi_mask(func_img_smooth)

model = FirstLevelModel(
    t_r=2.0,
    noise_model='ar1',
    standardize=True,
    hrf_model='spm',
    mask_img=mask_img
)

model = model.fit(func_img_z, events=events_df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import masking

def plot_voxel_fit(model, voxel_coords, func_img, mask_img, title="GLM Fit at Voxel"):
    """
    Plot actual vs predicted BOLD signal for a specific voxel (x, y, z) using a provided mask.
    """
    x, y, z = voxel_coords


    mask_data = mask_img.get_fdata().astype(bool)


    if not mask_data[x, y, z]:
        raise ValueError(f"Voxel {voxel_coords} is outside the mask")


    voxel_index_3d = np.ravel_multi_index((x, y, z), mask_data.shape)


    mask_flat = mask_data.flatten()
    masked_indices = np.where(mask_flat)[0]

    try:
        masked_index = np.where(masked_indices == voxel_index_3d)[0][0]
    except IndexError:
        raise ValueError(f"Voxel {voxel_coords} is not in the brain mask.")


    actual = masking.apply_mask(func_img, mask_img)[:, masked_index]
    predicted = model.predicted[0][:, masked_index]


    plt.figure(figsize=(10, 4))
    plt.plot(model.frame_times, actual, label='Observed BOLD', marker='o')
    plt.plot(model.frame_times, predicted, label='GLM Prediction', linestyle='--')
    plt.xlabel("Time (s)")
    plt.ylabel("Signal")
    plt.title(f"{title} at voxel {voxel_coords} (mask index {masked_index})")
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
model = FirstLevelModel(
    t_r=2.0,
    noise_model='ar1',
    standardize=True,
    hrf_model='spm',
    mask_img=mask_img,
    minimize_memory=False
)

model = model.fit(func_img_z, events=pd.read_csv(event_path, sep='\t'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import masking

def plot_voxel_fit2(model, voxel_coords, func_img, mask_img, title="GLM Fit at Voxel"):
    """
    Plot actual vs predicted BOLD signal for a specific voxel (x, y, z).
    All image objects are accessed safely via .get_fdata().
    """
    x, y, z = voxel_coords

    # Convert NIfTI image to NumPy array
    mask_data = mask_img.get_fdata().astype(bool)


    try:
        if not mask_data[x, y, z]:
            raise ValueError(f"Voxel {voxel_coords} is outside the mask")
    except IndexError:
        raise ValueError(f"Voxel {voxel_coords} is outside image bounds")

    flat_index = np.ravel_multi_index((x, y, z), mask_data.shape)
    masked_voxel_indices = np.where(mask_data.flatten())[0]

    try:
        masked_index = np.where(masked_voxel_indices == flat_index)[0][0]
    except IndexError:
        raise ValueError(f"Voxel {voxel_coords} not found in the mask")


    actual_ts = masking.apply_mask(func_img, mask_img)[:, masked_index]
    predicted_ts = model.predicted[0][:, masked_index]

    plt.figure(figsize=(10, 4))
    plt.plot(model.frame_times, actual_ts, label="Observed BOLD", marker='o')
    plt.plot(model.frame_times, predicted_ts, label="Predicted", linestyle='--')
    plt.xlabel("Time (s)")
    plt.ylabel("Signal")
    plt.title(f"{title} at voxel {voxel_coords} (mask idx {masked_index})")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
plot_voxel_fit2(
    model=model,
    voxel_coords=(30, 50, 40),
    func_img=func_img_z,
    mask_img=mask_img
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import masking

def plot_voxel_fit4(model, voxel_coords, func_img, mask_img, title="GLM Fit at Voxel"):
    """
    Plot actual vs predicted BOLD signal for a specific voxel (x, y, z).
    Ensures all objects are NumPy arrays before any indexing.
    """
    x, y, z = voxel_coords


    if hasattr(mask_img, "get_fdata"):
        mask_data = mask_img.get_fdata().astype(bool)
    else:
        mask_data = mask_img.astype(bool)

    print("Type of mask_data:", type(mask_data))

    try:
        if not mask_data[x, y, z]:
            raise ValueError(f"Voxel {voxel_coords} is outside the brain mask.")
    except TypeError as e:
        print("mask_data IS STILL A NIfTI OBJECT SOMEHOW")
        raise e


    observed_matrix = masking.apply_mask(func_img, mask_img)
    predicted_matrix = model.predicted[0]


    flat_index = np.ravel_multi_index((x, y, z), mask_data.shape)
    all_masked_indices = np.where(mask_data.flatten())[0]

    try:
        masked_index = np.where(all_masked_indices == flat_index)[0][0]
    except IndexError:
        raise ValueError(f"Voxel {voxel_coords} not found in mask.")


    plt.figure(figsize=(10, 4))
    plt.plot(model.design_matrices_[0].index.values, observed_matrix[:, masked_index], label='Observed BOLD', marker='o')
    plt.plot(model.design_matrices_[0].index.values, predicted_matrix[:, masked_index], label='Predicted BOLD', linestyle='--')
    plt.xlabel("Time (s)")
    plt.ylabel("BOLD Signal")
    plt.title(f"{title} at voxel {voxel_coords} (mask index {masked_index})")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
plot_voxel_fit4(model, voxel_coords=(30, 50, 40), func_img=func_img_z, mask_img=mask_img)
